In [5]:
import cv2
import math
import numpy as np
import time

camera = cv2.VideoCapture(0)

cx=0
cy=0
while True:    
    success, frame = camera.read()  
    if success:    
        height, width = frame.shape[0:2]  
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)    
        bin1 = cv2.inRange(hsv, (17, 158, 0), (35, 255, 255)) # желтый
        bin2 = cv2.inRange(hsv, (0, 158, 0), (16, 255, 255)) # красный
        bin3 = cv2.inRange(hsv, (36, 158, 0), (52, 255, 255)) # зеленый
        bin4 = cv2.inRange(hsv, (66, 158, 0), (108, 255, 255)) # голубой
        binary = bin1+bin2+bin3+bin4  
        roi = cv2.bitwise_and(frame, frame, mask=binary)   
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_NONE) 
        for i,cnt in enumerate(contours):
            if len(contours) != 0:  
                maxc = max(contours, key=cv2.contourArea)  
                moments = cv2.moments(cnt) 
                if moments["m00"] > 20: 
                    cx = int(moments["m10"] / moments["m00"])  
                    cy = int(moments["m01"] / moments["m00"])  
                    cv2.drawContours(frame, maxc, -1, (0, 255, 0), 2) 
                    cv2.line(frame, (cx, 0), (cx, height), (0, 255, 0), 2)  
                    cv2.line(frame, (0, cy), (width, cy), (0, 255, 0), 2)  
            cv2.putText(frame, 'Axes x: {}'.format(cx), (width - 500, height - 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
            cv2.putText(frame, 'Axes y: {}'.format(cy), (width - 500, height - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
        cv2.imshow('frame', frame)  # выводим все кадры на экран
        cv2.waitKey(1)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2
import math
import numpy as np
import time
import pyrealsense2
from realsense_camera import *

camera = RealsenseCamera()

cx=0
cy=0
cx_mm=0
cy_mm=0
Cx=320
Cy=240
fx=604.602
fy=604.162

def getOrientation(pts, img):
    rect = cv2.minAreaRect(pts)
    width = int(rect[1][0])
    height = int(rect[1][1])
    angle = int(rect[2])
    if width < height:
        angle = 90- angle
    else:
        angle = 90+(90-angle)
    return angle,width, height


while True:
    success, depth_colormap, frame, depth = camera.get_frame_stream()
    if success:    
        height, width = frame.shape[0:2]   
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)    
        bin1 = cv2.inRange(hsv, (17, 158, 0), (35, 255, 255)) # желтый
        bin2 = cv2.inRange(hsv, (0, 118, 101), (16, 196, 163)) # красный
        bin3 = cv2.inRange(hsv, (36, 158, 0), (52, 255, 255)) # зеленый
        bin4 = cv2.inRange(hsv, (66, 158, 0), (108, 255, 255)) # голубой
        binary = bin1+bin2+bin3+bin4  
        binary = bin2
        roi = cv2.bitwise_and(frame, frame, mask=binary)   
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_NONE) 
        for i,cnt in enumerate(contours):
            if len(contours) != 0:  
                maxc = max(contours, key=cv2.contourArea)  
                moments = cv2.moments(cnt) 
                if moments["m00"] > 20: 
                    cx = int(moments["m10"] / moments["m00"])  
                    cy = int(moments["m01"] / moments["m00"])  
                    cx_mm=depth[cy,cx]*((Cx-cx)/fx)
                    cy_mm=depth[cy,cx]*((Cy-cy)/fy)                  
                    rotation,width,height=getOrientation(maxc, frame)
                    cv2.drawContours(frame, maxc, -1, (0, 255, 0), 2) 
                    cv2.line(frame, (cx, 0), (cx, height), (0, 255, 0), 2)  
                    cv2.line(frame, (0, cy), (width, cy), (0, 255, 0), 2)  
            cv2.putText(frame, 'Axes x: {}'.format(cx), (width - 500, height - 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
            cv2.putText(frame, 'Axes y: {}'.format(cy), (width - 500, height - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
        cv2.imshow('frame', frame)  # выводим все кадры на экран
        cv2.waitKey(1)
cv2.destroyAllWindows()

In [4]:
import cv2
import math
import numpy as np
import time

camera = cv2.VideoCapture(0)

cx=0
cy=0
rotation=0
def getOrientation(pts, img):
    rect = cv2.minAreaRect(pts)
    width = int(rect[1][0])
    height = int(rect[1][1])
    angle = int(rect[2])
    if width < height:
        angle = 90- angle
    else:
        angle = 90+(90-angle)
    return angle,width, height

while True:    
    success, frame = camera.read()  
    if success:    
        height, width = frame.shape[0:2]  
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)    
        bin1 = cv2.inRange(hsv, (17, 158, 0), (35, 255, 255)) # желтый
        bin2 = cv2.inRange(hsv, (0, 158, 0), (16, 255, 255)) # красный
        bin3 = cv2.inRange(hsv, (36, 158, 0), (52, 255, 255)) # зеленый
        bin4 = cv2.inRange(hsv, (66, 158, 0), (108, 255, 255)) # голубой
        binary = bin1+bin2+bin3+bin4  
        roi = cv2.bitwise_and(frame, frame, mask=binary)   
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_NONE) 
        for i,cnt in enumerate(contours):
            if len(contours) != 0:  
                maxc = max(contours, key=cv2.contourArea)  
                moments = cv2.moments(cnt) 
                if moments["m00"] > 20: 
                    cx = int(moments["m10"] / moments["m00"])  
                    cy = int(moments["m01"] / moments["m00"])  
                    rotation,width,height=getOrientation(maxc, frame)
                    cv2.drawContours(frame, maxc, -1, (0, 255, 0), 2) 
                    cv2.line(frame, (cx, 0), (cx, height), (0, 255, 0), 2)  
                    cv2.line(frame, (0, cy), (width, cy), (0, 255, 0), 2) 
            cv2.putText(frame, 'rotation {}'.format(rotation), (width - 500, height - 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
            cv2.putText(frame, 'Axes x: {}'.format(cx), (width - 500, height - 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
            cv2.putText(frame, 'Axes y: {}'.format(cy), (width - 500, height - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA) 
        cv2.imshow('frame', frame)  # выводим все кадры на экран
        cv2.waitKey(1)
cv2.destroyAllWindows()

KeyboardInterrupt: 